# Valuation with Inferred FX Rates

This notebooks shows how to value a portfolio using recipes with FX rates inferred and not explicitly loaded into the quotes store.

- Instruments are priced in GBP.
- LUSID performs a valuation in JPY by inverting the provided JPY/GBP FX rate.
- LUSID then performs a valuation in AUD by triangulating the provided GBP/USD and AUD/USD FX rates.


## 1. Setup LUSID

In [1]:
# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import os
import pandas as pd
import json
import pytz
from datetime import datetime

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [2]:
# define the apis
aggregation_api = api_factory.build(lusid.AggregationApi)
quotes_api = api_factory.build(lusid.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
instruments_api = api_factory.build(lusid.InstrumentsApi)
transaction_portfolios_api = api_factory.build(lusid.TransactionPortfoliosApi)

In [3]:
# Defining variables
valuation_date = datetime(year=2022, month=3, day=8, tzinfo=pytz.UTC)
valuation_date_later = datetime(year=2022, month=3, day=8, minute=1, tzinfo=pytz.UTC)
scope = "FX_Conversion_NB"
portfolio_code = "FX_Conversion_NB"

## 2. Create Portfolio

In [4]:
try: 
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name="FX_Conversion_NB",
            code=portfolio_code,
            created="2020-01-01T00:00:00+00:00",
            base_currency="GBP"
        )
    )
    
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'FX_Conversion_NB' because it already exists in scope 'FX_Conversion_NB'.


## 3. Load Instruments

Create 5 instruments with unique 'ClientInternal' identifiers.

In [5]:
for i in range(1, 6):
    instrument_body = models.InstrumentDefinition(
        name=f"instrument_{i}",
        identifiers={"ClientInternal": models.InstrumentIdValue(value=f"client_internal_{i}")}
    )
    
    reponse = instruments_api.upsert_instruments(
        request_body={
            "request_id_1": instrument_body
        }
    )

## 4. Load Quotes

### 4.1 Instrument Value Quotes

Load a selection of prices for the 5 instruments into the quotes store with the same 'ClientInternal' identifiers as those of the instruments created.

In [6]:
values = [20, 50, 100, 75, 50]

for i in range(1, 6):
    quotes_api.upsert_quotes(
        scope = scope,
        request_body = {f"quote_{i}": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=f"client_internal_{i}",
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid"
                
                ),
                effective_at = valuation_date
            ),
            metric_value=models.MetricValue(
                value=values[i-1],
                unit="GBP"
            )
        )}
        
    )

### 4.2 FX Rate Quotes

Load FX rates required for FX rate inference into the quotes store.

In [7]:
def spot_request(from_ccy, to_ccy, rate, valuation_date):
            return models.UpsertQuoteRequest(
                       quote_id=models.QuoteId(
                           models.QuoteSeriesId(
                               provider='Lusid',
                               instrument_id=f'{from_ccy}/{to_ccy}',
                               instrument_id_type='CurrencyPair',
                               quote_type='Rate',
                               field='mid'
                           ),
                           effective_at=valuation_date
                       ),
                       metric_value=models.MetricValue(
                           value=rate,
                           unit=f'{from_ccy}/{to_ccy}'
                       ),
                       lineage='None'
            )

#### JYP/GBP FX Rate Quote

Load JPY/GBP FX rate into the quotes store for the valuation date.

In [8]:
response = quotes_api.upsert_quotes(scope=scope,
                                   request_body={"1": spot_request("JPY", "GBP", 0.006618, valuation_date)})

# display(lusid_response_to_data_frame(response))

#### GBP/USD FX Rate Quote

Load GBP/USD FX rate into the quotes store for the valuation date.

In [9]:
response = quotes_api.upsert_quotes(scope=scope,
                                   request_body={"1": spot_request("GBP", "USD", 1.3106, valuation_date)})

# display(lusid_response_to_data_frame(response))

#### AUD/USD FX Rate Quote

Load AUD/USD FX rate into the quotes store for the valuation date.

In [10]:
response = quotes_api.upsert_quotes(scope=scope,
                                   request_body={"1": spot_request("AUD", "USD", 0.7319, valuation_date_later)})

# display(lusid_response_to_data_frame(response))

## 5. Set Holdings

Set holdings of 1 unit for each of the 5 instruments in the portfolio.

In [11]:
for i in range(1, 6):
    response = transaction_portfolios_api.adjust_holdings(
        scope = scope,
        code = portfolio_code,
        effective_at = valuation_date,
        adjust_holding_request = [
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    "instrument/default/ClientInternal": f"client_internal_{i}"
                },
                tax_lots= [
                    models.TargetTaxLotRequest(
                        units=1
                    )
                ]
            )
        ]
    )

## 6. Valuation

Complete a valuation on the instruments with inferred FX rates.

### 6.1 Valuation Recipe

Define a valuation recipe to use the quotes loaded into the quotes store. 

*** Important *** - The attempt_to_infer_missing_fx option is set to True on line 36 below enabling LUSID to attempt to determine missing FX rates from rates in the quotes store.

In [12]:
# Create recipes
recipe_scope="FX_Conversion_NB"
recipe_code="FX_Conversion_NB"


# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            models.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
            ),
            models.MarketDataKeyRule(
                key='Fx.CurrencyPair.*',
                data_scope=scope,
                supplier='Lusid',
                quote_type='Rate',
                quote_interval='1D.0D',
                field="mid"
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope='Lusid',
        ### IMPORTANT ###
        # This enables FX rate inference
        #------------------------------------
            attempt_to_infer_missing_fx=True,
#             save_inferred_missing_fx=True,
        #------------------------------------
        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
    ),
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

### 6.2 Valuation Request

Create valuation requests for the holdings with JPY & AUD report currencies (no explicit FX rates from GBP to these in the quotes store). 

In [13]:
def generate_valuation_request(valuation_effectiveAt: datetime, report_currency: str):

    # Create the valuation request
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(
            scope=recipe_scope, code=recipe_code
        ),
        metrics=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Value"),
            models.AggregateSpec("Valuation/PvInReportCcy/Ccy", "Value"),
            models.AggregateSpec("Valuation/PvInPortfolioCcy", "Value"),
            models.AggregateSpec("Valuation/PvInPortfolioCcy/Ccy", "Value"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Proportion"),
            models.AggregateSpec("Holding/Units", "Value"),
            models.AggregateSpec("Aggregation/Errors", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        report_currency = report_currency,
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(
            effective_at=valuation_effectiveAt.isoformat()
        ),
    )

    return valuation_request

### 6.3 Valuation - Infering FX Rate - Inversion

The explicit GBP/JPY FX rate does not exist in the quotes store and so LUSID inverts the JPY/GBP FX rate via the API call. This functionality is enabled by setting "attempt_to_infer_missing_fx" to true in the valuation recipe (see above).

In [14]:
aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(
        valuation_date, "JPY"
    )
)

output = pd.DataFrame(aggregation.data)
output

,Valuation/PvInReportCcy/Ccy,Valuation/PvInPortfolioCcy/Ccy,Instrument/default/Name,Valuation/PvInReportCcy,Valuation/PvInPortfolioCcy,Proportion(Valuation/PvInReportCcy),Holding/Units,Aggregation/Errors
0,JPY,GBP,instrument_5,7555.152614,50.0,0.169492,1.0,[]
1,JPY,GBP,instrument_4,11332.728921,75.0,0.254237,1.0,[]
2,JPY,GBP,instrument_3,15110.305228,100.0,0.338983,1.0,[]
3,JPY,GBP,instrument_2,7555.152614,50.0,0.169492,1.0,[]
4,JPY,GBP,instrument_1,3022.061046,20.0,0.067797,1.0,[]


### 6.4 Valuation - Infering FX Rate - Triangulation

The explicit GBP/AUD FX rate does not exist in the quotes store and so LUSID triangulates it from the GBP/USD and AUD/USD FX rates. This functionality is enabled by setting "attempt_to_infer_missing_fx" to true in the valuation recipe (see above).

In [15]:
aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(
        valuation_date_later, "AUD"
    )
)

output = pd.DataFrame(aggregation.data)
output

,Valuation/PvInReportCcy/Ccy,Valuation/PvInPortfolioCcy/Ccy,Instrument/default/Name,Valuation/PvInReportCcy,Valuation/PvInPortfolioCcy,Proportion(Valuation/PvInReportCcy),Holding/Units,Aggregation/Errors
0,AUD,GBP,instrument_5,89.534089,50.0,0.169492,1.0,[]
1,AUD,GBP,instrument_4,134.301134,75.0,0.254237,1.0,[]
2,AUD,GBP,instrument_3,179.068179,100.0,0.338983,1.0,[]
3,AUD,GBP,instrument_2,89.534089,50.0,0.169492,1.0,[]
4,AUD,GBP,instrument_1,35.813636,20.0,0.067797,1.0,[]
